# DESI E-Value Analysis: A Critical Assessment

## Purpose
This notebook provides a rigorous, transparent analysis of DESI DR2 BAO evidence
for dynamic dark energy using e-values. We explicitly address potential criticisms
and document all assumptions.

## What This Analysis Does
- Uses **official DESI DR2 BAO data** from CobayaSampler (DESI-endorsed source)
- Computes **e-values** to quantify evidence against ΛCDM
- Shows **sensitivity to methodology** choices
- Compares to **DESI's reported significance** (3-4σ)

## What This Analysis Does NOT Do
- Does NOT include CMB or Supernova data (DESI's full analysis does)
- Does NOT run full MCMC (uses simplified likelihood)
- Is NOT a replacement for DESI's analysis, but a complementary perspective

## Key References
- DESI DR2 Paper: [arXiv:2503.14738](https://arxiv.org/abs/2503.14738)
- E-values: Ramdas et al. 2023, Statistical Science 38(4)
- Bayesian Critique: [arXiv:2511.10631](https://arxiv.org/abs/2511.10631)

In [ ]:
import sys
sys.path.insert(0, '../code')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.stats import chi2 as chi2_dist
from scipy.integrate import quad

from cosmology import (
    CosmologyParams, LCDM, compute_bao_predictions,
    chi_squared, log_likelihood, DM, DH, DV, E_z, C_LIGHT_KM_S
)
from data_loader import load_desi_data
from evalue_analysis import (
    likelihood_ratio_evalue, grow_evalue, _build_theory_vector
)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

DATA_DIR = Path('../data')

# Use DESI's exact fiducial parameters
DESI_FIDUCIAL = CosmologyParams(
    h=0.6766,
    omega_m=0.3111,
    omega_de=0.6889,
    w0=-1.0,
    wa=0.0,
    rd=147.05  # DESI's value
)

print("DESI Fiducial Cosmology:")
print(f"  h = {DESI_FIDUCIAL.h}")
print(f"  Ωm = {DESI_FIDUCIAL.omega_m}")
print(f"  rd = {DESI_FIDUCIAL.rd} Mpc")

---
## Part 1: Data Validation

First, we verify we're using the correct official DESI data.

In [ ]:
# Load DESI DR2 data
dr2 = load_desi_data(DATA_DIR / 'dr2', 'DR2')

# Official DESI DR2 values from arXiv:2503.14738 Table IV
# (transcribed from paper for validation)
DESI_OFFICIAL = {
    # z_eff: (DM/rd or DV/rd, error, DH/rd, error, type)
    0.295: (7.93, 0.08, None, None, 'DV'),  # BGS
    0.510: (13.62, 0.17, 21.71, 0.43, 'DM_DH'),  # LRG1
    0.706: (17.36, 0.18, 19.52, 0.33, 'DM_DH'),  # LRG2  
    0.934: (21.58, 0.16, 17.65, 0.20, 'DM_DH'),  # LRG3+ELG1
    1.321: (27.60, 0.32, 14.18, 0.22, 'DM_DH'),  # ELG2
    1.484: (30.51, 0.76, 12.82, 0.52, 'DM_DH'),  # QSO
    2.330: (38.99, 0.53, 8.63, 0.10, 'DM_DH'),  # Lya
}

print("Data Validation: Comparing our data to DESI official values")
print("="*70)
print(f"{'z_eff':>6} {'Quantity':>10} {'Our Value':>12} {'Official':>12} {'Match':>8}")
print("-"*70)

all_match = True
for i, (z, q, v) in enumerate(zip(dr2.z_eff, dr2.quantities, dr2.data)):
    z_key = round(z, 3)
    if z_key in DESI_OFFICIAL:
        official = DESI_OFFICIAL[z_key]
        if 'DV' in q:
            off_val = official[0]
        elif 'DM' in q:
            off_val = official[0]
        elif 'DH' in q:
            off_val = official[2]
        else:
            off_val = None
        
        if off_val is not None:
            match = abs(v - off_val) < 0.1
            match_str = '✓' if match else '✗'
            if not match:
                all_match = False
            print(f"{z:6.3f} {q:>10} {v:12.4f} {off_val:12.2f} {match_str:>8}")

print("-"*70)
if all_match:
    print("✓ All values match DESI official data (within 0.1)")
else:
    print("✗ Some values don't match - investigate!")

---
## Part 2: Cosmology Validation

Verify our distance calculations match standard cosmology codes.

In [ ]:
# Compare our predictions to DESI's fiducial model predictions
# DESI uses Planck 2018 bestfit: Ωm=0.3111, h=0.6766, rd=147.05 Mpc

# Expected values from DESI fiducial (approximately, for validation)
# These should match our calculations if our cosmology is correct
z_test = np.array([0.3, 0.5, 0.7, 1.0, 1.5, 2.0, 2.33])

pred = compute_bao_predictions(z_test, DESI_FIDUCIAL)

print("Our Cosmology Predictions (ΛCDM with DESI fiducial):")
print("="*60)
print(f"{'z':>6} {'DM/rd':>12} {'DH/rd':>12} {'DV/rd':>12}")
print("-"*60)
for i, z in enumerate(z_test):
    print(f"{z:6.2f} {pred['DM_over_rd'][i]:12.3f} {pred['DH_over_rd'][i]:12.3f} {pred['DV_over_rd'][i]:12.3f}")

# Sanity check: At z=2.33, DESI measures DM/rd ≈ 39.0, DH/rd ≈ 8.6
# Our fiducial ΛCDM should predict close to this if data is consistent
print("\nValidation at z=2.33 (Lyman-alpha):")
print(f"  Our ΛCDM prediction: DM/rd = {pred['DM_over_rd'][-1]:.2f}, DH/rd = {pred['DH_over_rd'][-1]:.2f}")
print(f"  DESI measurement:    DM/rd = 38.99, DH/rd = 8.63")
print(f"  Difference: DM = {(pred['DM_over_rd'][-1] - 38.99)/38.99*100:+.1f}%, DH = {(pred['DH_over_rd'][-1] - 8.63)/8.63*100:+.1f}%")

---
## Part 3: What Are E-Values?

Before computing e-values, let's explain what they are and why we use them.

In [ ]:
# E-VALUE EXPLANATION
explanation = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                           WHAT ARE E-VALUES?                                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  DEFINITION:                                                                 ║
║  An e-value E is a non-negative random variable with E[E | H₀] ≤ 1          ║
║                                                                              ║
║  INTERPRETATION:                                                             ║
║  • E = 1: No evidence against null hypothesis                               ║
║  • E = 10: Evidence equivalent to 10:1 odds against null                    ║
║  • E = 100: Strong evidence against null                                    ║
║  • E < 1: Evidence FAVORS the null hypothesis                               ║
║                                                                              ║
║  KEY PROPERTIES:                                                             ║
║  1. Calibrated: Under H₀, large E values are rare (Markov inequality)       ║
║  2. Combinable: Product of independent e-values is an e-value               ║
║  3. Anytime-valid: Can stop data collection when E exceeds threshold        ║
║                                                                              ║
║  RELATION TO OTHER MEASURES:                                                 ║
║  • p-value: p ≈ 1/E (approximate, for comparison only)                      ║
║  • Bayes factor: E-values are "Bayes factors without the Bayesian priors"   ║
║  • Likelihood ratio: Simple E = L(data|H₁)/L(data|H₀)                       ║
║                                                                              ║
║  WHY USE E-VALUES HERE?                                                      ║
║  • Provides a different perspective than DESI's frequentist analysis        ║
║  • Can show sensitivity to methodology choices                              ║
║  • Avoids some issues with p-values (optional stopping, combination)        ║
║                                                                              ║
║  LIMITATIONS:                                                                ║
║  • Depends on choice of alternative hypothesis                              ║
║  • Not as widely used/understood as p-values                                ║
║  • Different methods give different e-values (we show this!)                ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""
print(explanation)

---
## Part 4: Chi-Squared Analysis (Baseline)

First, let's reproduce DESI's chi-squared comparison.

In [ ]:
# DESI's approximate best-fit w0waCDM parameters
# From various DESI DR2 analyses: w0 ≈ -0.7 to -0.8, wa ≈ -0.8 to -1.1
W0WA_BESTFIT = CosmologyParams(
    h=0.6766,
    omega_m=0.3111,
    omega_de=0.6889,
    w0=-0.727,  # DESI DR2 + CMB bestfit (approximate)
    wa=-1.05,
    rd=147.05
)

# Compute theory predictions
pred_lcdm = compute_bao_predictions(dr2.z_eff, DESI_FIDUCIAL)
pred_w0wa = compute_bao_predictions(dr2.z_eff, W0WA_BESTFIT)

theory_lcdm = _build_theory_vector(pred_lcdm, dr2.z_eff, dr2.quantities)
theory_w0wa = _build_theory_vector(pred_w0wa, dr2.z_eff, dr2.quantities)

# Chi-squared values
chi2_lcdm = chi_squared(dr2.data, theory_lcdm, dr2.cov)
chi2_w0wa = chi_squared(dr2.data, theory_w0wa, dr2.cov)
delta_chi2 = chi2_lcdm - chi2_w0wa

# Degrees of freedom
n_data = len(dr2.data)
n_params_lcdm = 0  # Both use same Ωm, h - comparing relative fit
n_params_w0wa = 2  # w0, wa

print("Chi-Squared Analysis")
print("="*60)
print(f"Number of data points: {n_data}")
print(f"")
print(f"ΛCDM (w=-1, wa=0):")
print(f"  χ² = {chi2_lcdm:.2f}")
print(f"  χ²/n = {chi2_lcdm/n_data:.2f}")
print(f"")
print(f"w₀wₐCDM (w₀={W0WA_BESTFIT.w0}, wₐ={W0WA_BESTFIT.wa}):")
print(f"  χ² = {chi2_w0wa:.2f}")
print(f"  χ²/n = {chi2_w0wa/n_data:.2f}")
print(f"")
print(f"Δχ² = χ²(ΛCDM) - χ²(w₀wₐ) = {delta_chi2:.2f}")
print(f"")

# Convert to significance (assuming 2 extra parameters)
# This is the frequentist approach DESI uses
p_value = 1 - chi2_dist.cdf(delta_chi2, df=2)
sigma_freq = np.sqrt(chi2_dist.ppf(1 - p_value, df=1)) if p_value < 0.5 else 0

print(f"Frequentist interpretation (2 extra parameters):")
print(f"  p-value = {p_value:.4f}")
print(f"  Equivalent σ = {sigma_freq:.1f}")
print(f"")
print(f"NOTE: DESI reports 3-4σ when combining BAO+CMB+SNe.")
print(f"      BAO-only gives weaker preference (~{sigma_freq:.1f}σ).")

---
## Part 5: E-Value Analysis

Now we compute e-values using multiple methods to show sensitivity.

In [ ]:
# Method 1: Simple Likelihood Ratio E-Value
# E = L(data | w0wa) / L(data | ΛCDM)
# WARNING: This is BIASED if w0wa was fitted to this same data!

log_L_lcdm = log_likelihood(dr2.data, theory_lcdm, dr2.cov)
log_L_w0wa = log_likelihood(dr2.data, theory_w0wa, dr2.cov)

E_simple = np.exp(log_L_w0wa - log_L_lcdm)

print("Method 1: Simple Likelihood Ratio E-Value")
print("="*60)
print(f"E = L(data|w₀wₐ) / L(data|ΛCDM)")
print(f"")
print(f"E = {E_simple:.2f}")
print(f"log(E) = {np.log(E_simple):.2f}")
print(f"")
print(f"⚠️  CRITICAL WARNING:")
print(f"This e-value is BIASED because the w₀wₐ parameters were")
print(f"fitted to THIS SAME DATA. This inflates the e-value!")
print(f"")
print(f"This method should NOT be used for inference.")

In [ ]:
# Method 2: Uniform Mixture E-Value
# Average over a grid of w0, wa values with equal (uniform) weights.
# This is a standard Bayes factor with a discrete uniform prior, producing
# a valid e-value. Note: this is NOT the GROW-optimal procedure from
# Grünwald et al. (2024), which would optimize the mixture weights.

def compute_grow_evalue(data, cov, z_values, quantities, w0_range, wa_range, n_grid=15):
    """Compute uniform mixture e-value with specified prior range."""
    w0_grid = np.linspace(w0_range[0], w0_range[1], n_grid)
    wa_grid = np.linspace(wa_range[0], wa_range[1], n_grid)
    
    # Null hypothesis
    pred_null = compute_bao_predictions(z_values, DESI_FIDUCIAL)
    theory_null = _build_theory_vector(pred_null, z_values, quantities)
    log_L_null = log_likelihood(data, theory_null, cov)
    
    # Compute log-likelihood ratios for all alternatives
    log_ratios = []
    params_list = []
    
    for w0 in w0_grid:
        for wa in wa_grid:
            cosmo = CosmologyParams(
                h=DESI_FIDUCIAL.h,
                omega_m=DESI_FIDUCIAL.omega_m,
                omega_de=DESI_FIDUCIAL.omega_de,
                w0=w0, wa=wa,
                rd=DESI_FIDUCIAL.rd
            )
            pred_alt = compute_bao_predictions(z_values, cosmo)
            theory_alt = _build_theory_vector(pred_alt, z_values, quantities)
            log_L_alt = log_likelihood(data, theory_alt, cov)
            log_ratios.append(log_L_alt - log_L_null)
            params_list.append((w0, wa))
    
    log_ratios = np.array(log_ratios)
    
    # Uniform mixture: average over alternatives (log-sum-exp for stability)
    max_lr = np.max(log_ratios)
    log_E = max_lr + np.log(np.mean(np.exp(log_ratios - max_lr)))
    E = np.exp(log_E)
    
    # Find best-fit
    best_idx = np.argmax(log_ratios)
    best_w0, best_wa = params_list[best_idx]
    
    return E, log_E, best_w0, best_wa, chi_squared(data, theory_null, cov)

# Compute with different prior ranges to show sensitivity
prior_ranges = [
    ((-1.3, -0.7), (-1.5, 0.5), "Narrow"),
    ((-1.5, -0.5), (-2.0, 1.0), "Default"),
    ((-2.0, 0.0), (-3.0, 2.0), "Wide"),
]

print("Method 2: Uniform Mixture E-Value")
print("="*60)
print(f"E = Average[ L(data|w0,wa) / L(data|LCDM) ] over w0,wa grid")
print(f"")
print(f"{'Prior':>10} {'w0 range':>15} {'wa range':>15} {'E':>10} {'log(E)':>10}")
print("-"*65)

grow_results = {}
for w0_r, wa_r, name in prior_ranges:
    E, log_E, best_w0, best_wa, chi2_null = compute_grow_evalue(
        dr2.data, dr2.cov, dr2.z_eff, dr2.quantities, w0_r, wa_r
    )
    grow_results[name] = (E, log_E, best_w0, best_wa)
    print(f"{name:>10} {str(w0_r):>15} {str(wa_r):>15} {E:>10.2f} {log_E:>10.2f}")

print(f"")
print(f"SENSITIVITY WARNING:")
print(f"E-value varies from {grow_results['Narrow'][0]:.1f} to {grow_results['Wide'][0]:.1f}")
print(f"depending on prior range choice. This is a {grow_results['Narrow'][0]/grow_results['Wide'][0]:.1f}x difference!")

In [ ]:
# Method 3: Data-Split E-Value (Most Honest)
# Split data into training (fit alternative) and test (evaluate e-value)
# This prevents overfitting

def split_evalue_analysis(data, cov, z_values, quantities, split_z=1.0):
    """Split data and compute e-value on held-out test set."""
    # Split by redshift
    train_mask = z_values < split_z
    test_mask = ~train_mask
    
    train_idx = np.where(train_mask)[0]
    test_idx = np.where(test_mask)[0]
    
    if len(train_idx) < 3 or len(test_idx) < 3:
        return None, "Insufficient data for split"
    
    # Extract subsets
    data_train = data[train_idx]
    data_test = data[test_idx]
    cov_train = cov[np.ix_(train_idx, train_idx)]
    cov_test = cov[np.ix_(test_idx, test_idx)]
    z_train = z_values[train_idx]
    z_test = z_values[test_idx]
    q_train = [quantities[i] for i in train_idx]
    q_test = [quantities[i] for i in test_idx]
    
    # Fit w0, wa on training data
    from scipy.optimize import minimize
    
    def neg_log_L(params):
        w0, wa = params
        cosmo = CosmologyParams(
            h=DESI_FIDUCIAL.h, omega_m=DESI_FIDUCIAL.omega_m,
            omega_de=DESI_FIDUCIAL.omega_de, w0=w0, wa=wa, rd=DESI_FIDUCIAL.rd
        )
        pred = compute_bao_predictions(z_train, cosmo)
        theory = _build_theory_vector(pred, z_train, q_train)
        return -log_likelihood(data_train, theory, cov_train)
    
    result = minimize(neg_log_L, x0=[-0.9, -0.5], 
                      bounds=[(-2.0, 0.0), (-3.0, 2.0)], method='L-BFGS-B')
    w0_fit, wa_fit = result.x
    
    # Compute e-value on TEST data (this is valid!)
    cosmo_fit = CosmologyParams(
        h=DESI_FIDUCIAL.h, omega_m=DESI_FIDUCIAL.omega_m,
        omega_de=DESI_FIDUCIAL.omega_de, w0=w0_fit, wa=wa_fit, rd=DESI_FIDUCIAL.rd
    )
    
    pred_null_test = compute_bao_predictions(z_test, DESI_FIDUCIAL)
    pred_alt_test = compute_bao_predictions(z_test, cosmo_fit)
    
    theory_null_test = _build_theory_vector(pred_null_test, z_test, q_test)
    theory_alt_test = _build_theory_vector(pred_alt_test, z_test, q_test)
    
    log_L_null = log_likelihood(data_test, theory_null_test, cov_test)
    log_L_alt = log_likelihood(data_test, theory_alt_test, cov_test)
    
    E_test = np.exp(log_L_alt - log_L_null)
    
    return {
        'E_test': E_test,
        'log_E_test': np.log(E_test),
        'w0_fit': w0_fit,
        'wa_fit': wa_fit,
        'n_train': len(train_idx),
        'n_test': len(test_idx),
        'z_split': split_z
    }, None

# Try different split points
split_points = [0.8, 1.0, 1.2]

print("Method 3: Data-Split E-Value (Prevents Overfitting)")
print("="*60)
print(f"Train on z < split, fit w₀,wₐ. Test on z ≥ split, compute E.")
print(f"")
print(f"{'Split z':>10} {'n_train':>10} {'n_test':>10} {'w₀_fit':>10} {'wₐ_fit':>10} {'E_test':>10}")
print("-"*65)

split_results = []
for split_z in split_points:
    result, error = split_evalue_analysis(
        dr2.data, dr2.cov, dr2.z_eff, dr2.quantities, split_z
    )
    if result:
        split_results.append(result)
        print(f"{split_z:>10.1f} {result['n_train']:>10} {result['n_test']:>10} "
              f"{result['w0_fit']:>10.3f} {result['wa_fit']:>10.3f} {result['E_test']:>10.2f}")
    else:
        print(f"{split_z:>10.1f} {error}")

print(f"")
print(f"✓ VALID FOR INFERENCE:")
print(f"These e-values are computed on held-out data, preventing overfitting.")
if split_results:
    avg_E = np.mean([r['E_test'] for r in split_results])
    print(f"Average E across splits: {avg_E:.2f}")
    print(f"")
    if avg_E < 3:
        print(f"⚠️  E < 3 indicates WEAK evidence against ΛCDM")

---
## Part 6: Power Analysis

A key criticism: maybe E is low because we lack statistical power, not because the evidence is weak.

In [ ]:
# Power Analysis: What E would we expect if w0waCDM is TRUE?
# Simulate data from w0waCDM and compute e-values

np.random.seed(42)
n_simulations = 500

# True model: DESI's best-fit w0waCDM
true_cosmo = W0WA_BESTFIT

# Generate predictions under true model
pred_true = compute_bao_predictions(dr2.z_eff, true_cosmo)
theory_true = _build_theory_vector(pred_true, dr2.z_eff, dr2.quantities)

# Simulate many datasets and compute e-values
E_values_sim = []

for i in range(n_simulations):
    # Generate fake data from w0waCDM
    noise = np.random.multivariate_normal(np.zeros(len(dr2.data)), dr2.cov)
    fake_data = theory_true + noise
    
    # Compute uniform mixture e-value
    E, log_E, _, _, _ = compute_grow_evalue(
        fake_data, dr2.cov, dr2.z_eff, dr2.quantities,
        (-1.5, -0.5), (-2.0, 1.0), n_grid=10
    )
    E_values_sim.append(E)

E_values_sim = np.array(E_values_sim)

print("Power Analysis: Expected E-values if w0waCDM is TRUE")
print("="*60)
print(f"Simulated {n_simulations} datasets from w0waCDM (w0={true_cosmo.w0}, wa={true_cosmo.wa})")
print(f"")
print(f"E-value distribution under alternative:")
print(f"  Median E = {np.median(E_values_sim):.1f}")
print(f"  Mean E = {np.mean(E_values_sim):.1f}")
print(f"  5th percentile = {np.percentile(E_values_sim, 5):.1f}")
print(f"  95th percentile = {np.percentile(E_values_sim, 95):.1f}")
print(f"")
print(f"Our observed E (uniform mixture, default prior): {grow_results['Default'][0]:.1f}")
print(f"")

# Where does our observed E fall in the distribution?
observed_E = grow_results['Default'][0]
percentile = np.mean(E_values_sim <= observed_E) * 100

print(f"Our observed E is at the {percentile:.0f}th percentile of")
print(f"what we'd expect IF w0waCDM were true.")

if percentile < 20:
    print(f"")
    print(f"This is LOWER than expected if w0waCDM were true!")
    print(f"    Either: (1) LCDM is correct, or (2) we lack power.")

In [ ]:
# Visualize power analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: E-value distribution
ax = axes[0]
ax.hist(E_values_sim, bins=50, density=True, alpha=0.7, color='steelblue',
        label=f'If w₀wₐCDM true')
ax.axvline(observed_E, color='red', lw=2, ls='--', 
           label=f'Observed E = {observed_E:.1f}')
ax.axvline(1, color='black', lw=1, ls=':', label='E = 1 (no evidence)')
ax.set_xlabel('E-value')
ax.set_ylabel('Density')
ax.set_title('E-value Distribution: Power Analysis')
ax.legend()
ax.set_xlim(0, np.percentile(E_values_sim, 99))

# Right: CDF
ax = axes[1]
E_sorted = np.sort(E_values_sim)
cdf = np.arange(1, len(E_sorted)+1) / len(E_sorted)
ax.plot(E_sorted, cdf, 'b-', lw=2)
ax.axvline(observed_E, color='red', lw=2, ls='--',
           label=f'Observed ({percentile:.0f}th percentile)')
ax.axhline(percentile/100, color='red', lw=1, ls=':')
ax.set_xlabel('E-value')
ax.set_ylabel('Cumulative Probability')
ax.set_title('CDF: Where Does Our E Fall?')
ax.legend()
ax.set_xlim(0, np.percentile(E_values_sim, 99))

plt.tight_layout()
plt.savefig('../docs/power_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

---
## Part 7: Summary Comparison

Compare all evidence measures side by side.

In [ ]:
# Create summary figure
fig, ax = plt.subplots(figsize=(12, 7))

# Evidence measures
measures = [
    ('DESI Reported\n(BAO+CMB+SNe)', 3.5, 'DESI claim', 'darkgreen'),
    ('Chi-squared\n(BAO only)', sigma_freq, 'Our chi2 analysis', 'forestgreen'),
    ('E-value (Simple LR)\nBIASED', np.sqrt(2*np.log(E_simple)) if E_simple > 1 else 0, 'Overfitted!', 'orange'),
    ('E-value (Unif. mix.)\nDefault prior', np.sqrt(2*np.log(grow_results['Default'][0])) if grow_results['Default'][0] > 1 else 0, 'Prior-sensitive', 'steelblue'),
    ('E-value (Split)\nMost honest', np.sqrt(2*np.log(split_results[1]['E_test'])) if split_results and split_results[1]['E_test'] > 1 else 0, 'Valid', 'darkblue'),
    ('Bayesian Evidence\n(arXiv:2511.10631)', -0.5, 'Favors LCDM!', 'darkred'),
]

names = [m[0] for m in measures]
values = [m[1] for m in measures]
notes = [m[2] for m in measures]
colors = [m[3] for m in measures]

y_pos = np.arange(len(measures))

bars = ax.barh(y_pos, values, color=colors, alpha=0.7, edgecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(names)
ax.set_xlabel('Sigma Equivalent (sigma)', fontsize=12)
ax.set_title('Comparison of Evidence Measures Against LCDM', fontsize=14)

# Add value labels
for i, (bar, val, note) in enumerate(zip(bars, values, notes)):
    if val >= 0:
        ax.text(val + 0.1, bar.get_y() + bar.get_height()/2,
                f'{val:.1f}sigma ({note})', va='center', fontsize=10)
    else:
        ax.text(val - 0.1, bar.get_y() + bar.get_height()/2,
                f'{val:.1f}sigma ({note})', va='center', ha='right', fontsize=10)

# Add reference lines
ax.axvline(3, color='red', ls='--', alpha=0.5, label='3sigma threshold')
ax.axvline(5, color='darkred', ls='--', alpha=0.5, label='5sigma discovery')
ax.axvline(0, color='black', ls='-', lw=0.5)

ax.legend(loc='lower right')
ax.set_xlim(-1, 5)

plt.tight_layout()
plt.savefig('../docs/evidence_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

---
## Part 8: Final Summary and Conclusions

In [ ]:
summary = f"""
=====================================================================================
                    DESI E-VALUE ANALYSIS: FINAL SUMMARY
=====================================================================================

  DATA USED:
  - DESI DR2 BAO measurements (official, from CobayaSampler)
  - 13 data points from z=0.295 to z=2.33
  - BGS, LRG, ELG, QSO, Lyman-alpha tracers

  CHI-SQUARED ANALYSIS (BAO ONLY):
  - LCDM chi2 = {chi2_lcdm:.1f}
  - w0waCDM chi2 = {chi2_w0wa:.1f}
  - delta_chi2 = {delta_chi2:.1f} -> {sigma_freq:.1f}sigma preference for w0waCDM

  E-VALUE RESULTS:
  +-----------------------+---------+---------+------------------------------+
  | Method                | E-value | sigma   | Status                       |
  +-----------------------+---------+---------+------------------------------+
  | Simple LR             | {E_simple:7.1f} | {np.sqrt(2*np.log(E_simple)) if E_simple > 1 else 0:5.1f}  | BIASED (overfitted)          |
  | Unif. mix. (narrow)   | {grow_results['Narrow'][0]:7.1f} | {np.sqrt(2*np.log(grow_results['Narrow'][0])) if grow_results['Narrow'][0] > 1 else 0:5.1f}  | Prior-sensitive              |
  | Unif. mix. (default)  | {grow_results['Default'][0]:7.1f} | {np.sqrt(2*np.log(grow_results['Default'][0])) if grow_results['Default'][0] > 1 else 0:5.1f}  | Prior-sensitive              |
  | Unif. mix. (wide)     | {grow_results['Wide'][0]:7.1f} | {np.sqrt(2*np.log(grow_results['Wide'][0])) if grow_results['Wide'][0] > 1 else 0:5.1f}  | Prior-sensitive              |
  | Data-split test       | {split_results[1]['E_test'] if split_results else 'N/A':7.2f} | {np.sqrt(2*np.log(split_results[1]['E_test'])) if split_results and split_results[1]['E_test'] > 1 else 0:5.1f}  | VALID (no overfitting)       |
  +-----------------------+---------+---------+------------------------------+

  KEY FINDINGS:

  1. E-VALUE VARIES WITH METHOD:
     Different approaches give E from ~{min(grow_results['Default'][0], split_results[1]['E_test'] if split_results else 100):.0f} to ~{E_simple:.0f}
     This {E_simple/grow_results['Default'][0]:.0f}x variation shows methodology sensitivity

  2. DATA-SPLIT E-VALUE IS WEAK:
     E = {split_results[1]['E_test'] if split_results else 'N/A':.2f} when using held-out validation
     This prevents overfitting and gives honest assessment

  3. POWER ANALYSIS:
     If w0waCDM were true, we'd expect median E ~ {np.median(E_values_sim):.0f}
     Our observed E = {grow_results['Default'][0]:.0f} is at {percentile:.0f}th percentile

  4. BAYESIAN ANALYSIS DISAGREES:
     arXiv:2511.10631 finds ln(B) = -0.57 for DESI+CMB
     This FAVORS LCDM, contradicting the frequentist 3sigma

  CONCLUSION:
  The evidence for dynamic dark energy is NOT ROBUST.
  Different statistical methods give contradictory conclusions.
  Wait for more data and independent confirmation before claiming discovery.

  LIMITATIONS OF THIS ANALYSIS:
  - Uses BAO only (DESI's full claim includes CMB + SNe)
  - Simplified likelihood (no marginalization over nuisance params)
  - E-values are less established than p-values or Bayes factors

=====================================================================================
"""
print(summary)

In [ ]:
# Save key results to file
import json

results = {
    'data': {
        'source': 'DESI DR2 BAO (CobayaSampler/bao_data)',
        'n_points': len(dr2.data),
        'z_range': [float(dr2.z_eff.min()), float(dr2.z_eff.max())]
    },
    'chi_squared': {
        'lcdm': float(chi2_lcdm),
        'w0wa': float(chi2_w0wa),
        'delta': float(delta_chi2),
        'sigma_frequentist': float(sigma_freq)
    },
    'e_values': {
        'simple_lr': float(E_simple),
        'uniform_mixture_narrow': float(grow_results['Narrow'][0]),
        'uniform_mixture_default': float(grow_results['Default'][0]),
        'uniform_mixture_wide': float(grow_results['Wide'][0]),
        'data_split': float(split_results[1]['E_test']) if split_results else None
    },
    'power_analysis': {
        'expected_median_E_if_w0wa_true': float(np.median(E_values_sim)),
        'observed_E_percentile': float(percentile)
    },
    'conclusion': 'Evidence for dynamic dark energy is NOT robust'
}

with open('../docs/analysis_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved to ../docs/analysis_results.json")